In [15]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
OPBG = [ 'código','age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein','glicemia']
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )
compare_dataframes(df1, df2)
df1.columns, df2.columns

Both CSV files have the same number of columns: 16
Column names are the same.
Column order is the same.
All common columns have the same data types.


(Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'),
 Index(['código', 'age at diagnosis (months)', 'ntbc dosis mg/kg/day',
        'ntbc levels (dbs)', 'sca (urine)', 'methionine (plasma)',
        'tyrosine (plasma)', 'phenylalanine (plasma)', 'pt (sec)', 'bili total',
        'gpt', 'got', 'ggt', 'alkaline phosphatase', 'alfa-fetoprotein',
        'glicemia'],
       dtype='object'))

In [16]:

binary_target = 'Alpha-Fet'
df_train, _ = extract_transform("data/tirosinemia.csv",                 OPBG,  ['alfa-fetoprotein','ntbc levels (dbs)','sca (urine)'],binary_target)
df_train.drop('código', axis=1, inplace=True)
df_test, df_test_raw  = extract_transform("data/tirosinemia_italia.csv", OPBG,  ['alfa-fetoprotein','ntbc levels (dbs)','sca (urine)'],binary_target)



In [17]:

Rome_mask = df_test_raw['código'].str.startswith('R')

df_test_FLOR  = encode_and_convert (df_test[~Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_ROME  = encode_and_convert (df_test[Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_FLOR['Alpha-Fet'] = df_test_FLOR['Alpha-Fet'].astype('int8').astype('category')
df_test_ROME['Alpha-Fet'] = df_test_ROME['Alpha-Fet'].astype('int8').astype('category')


compare_dataframes(df_test_ROME, df_test_FLOR)


Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


In [18]:


compare_dataframes(df_train, df_test_ROME)






Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


(make_a_study pid=3986690) [I 2023-06-02 20:15:11,820] A new study created in memory with name: glicemia


In [19]:

ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test_ROME)

In [20]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)
features

['age at diagnosis (months)',
 'ntbc dosis mg/kg/day',
 'ntbc levels (dbs)',
 'sca (urine)',
 'methionine (plasma)',
 'tyrosine (plasma)',
 'phenylalanine (plasma)',
 'pt (sec)',
 'bili total',
 'gpt',
 'got',
 'ggt',
 'alkaline phosphatase',
 'glicemia']

In [21]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=100,
    Independent_testset = True,
    Independent_testset_df = ray_df_test
)

studies = ray.get(studies)

(make_a_study pid=3986682) [I 2023-06-02 20:15:12,268] A new study created in memory with name: age at diagnosis (months)
(make_a_study pid=3986680) [I 2023-06-02 20:15:12,604] A new study created in memory with name: ntbc dosis mg/kg/day
(make_a_study pid=3986683) [I 2023-06-02 20:15:28,167] Trial 6 finished with values: [0.6191756272401434, 0.35824260115623474] and parameters: {'seed': 6759, 'max_depth': 6, 'eta': 0.45353759500018986, 'subsample': 0.2773242858830184, 'lambda': 0.4184107719972632, 'alpha': 0.6768646293385235, 'scale_pos_weight': 4.938862973492467, 'kfold_splits': 2}.  [repeated 19x across cluster]
(make_a_study pid=3986680) [I 2023-06-02 20:15:33,230] Trial 15 finished with values: [0.5017921146953406, 0.11937907338142395] and parameters: {'seed': 9316, 'max_depth': 6, 'eta': 0.4412032630206293, 'subsample': 0.40940798264580636, 'lambda': 0.5870578663991503, 'alpha': 0.22539017341056256, 'scale_pos_weight': 2.7298651772688114, 'kfold_splits': 2}.  [repeated 44x across

KeyboardInterrupt: 

(make_a_study pid=3986683) [I 2023-06-02 20:16:53,328] Trial 98 finished with values: [0.510752688172043, 0.12949253618717194] and parameters: {'seed': 5585, 'max_depth': 8, 'eta': 0.17265117584553194, 'subsample': 0.39641955933272977, 'lambda': 0.5556905854886565, 'alpha': 0.5666925310507819, 'scale_pos_weight': 1.4510145287673386, 'kfold_splits': 2}.  [repeated 37x across cluster]
(make_a_study pid=3986683) [I 2023-06-02 20:16:53,410] A new study created in memory with name: ntbc levels (dbs)
(make_a_study pid=3986690) [I 2023-06-02 20:16:56,078] A new study created in memory with name: sca (urine)
(make_a_study pid=3986680) [I 2023-06-02 20:16:58,336] Trial 94 finished with values: [0.3682795698924731, 0.35124680399894714] and parameters: {'seed': 8592, 'max_depth': 8, 'eta': 0.44110786750886727, 'subsample': 0.3227033519069556, 'lambda': 0.016475835394501814, 'alpha': 0.7155079416154675, 'scale_pos_weight': 0.8899017300179479, 'kfold_splits': 3}.  [repeated 33x across cluster]
(mak

In [ ]:
with open('aa.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    